In [ ]:
import sys
from tqdm import tqdm
import subprocess

def install_with_progress(package_file):
    """Установка и обновление пакетов с прогресс-баром"""
    print(f"📦 Обновляем зависимости из {package_file}...")
    
    # Читаем список пакетов
    try:
        with open(package_file, 'r') as f:
            packages = [line.strip() for line in f if line.strip() and not line.startswith('#')]
    except FileNotFoundError:
        print(f"❌ Файл {package_file} не найден. Пропускаем...")
        return
    
    print(f"🔍 Найдено {len(packages)} пакетов для обновления")
    
    # Обновляем пакеты с прогресс-баром
    for package in tqdm(packages, desc="📥 Обновление пакетов"):
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", package, "-q"], check=True, capture_output=True)
        except subprocess.CalledProcessError as e:
            print(f"⚠️ Ошибка при обновлении {package}: {e}")
    
    print("✅ Обновление завершено!")

# Обновляем зависимости из requirements.txt
install_with_progress("requirements.txt")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import json
import os
import numpy as np
import soundfile as sf
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.cuda.amp import autocast, GradScaler
from transformers import (
    AutoTokenizer,
    AutoConfig,
    BitsAndBytesConfig,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Model
)
from transformers.models.gemma3.configuration_gemma3 import Gemma3TextConfig
from transformers.models.gemma3.modeling_gemma3 import Gemma3ForCausalLM
from huggingface_hub import login
import jiwer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
from huggingface_hub import notebook_login
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR
import zipfile
import io
import wandb
import glob
import re
import random
import itertools
from IPython.display import Audio, display

def set_seed(seed=42):
    """Устанавливает random seed для всех нужных библиотек."""
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # Обеспечивает детерминированность, но может замедлить обучение
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"🌱 Random Seed установлен: {seed}")

# Устанавливаем seed для воспроизводимости
set_seed(42)

In [ ]:
# !nvidia-smi -l 1

In [ ]:
notebook_login()

In [ ]:
def to_bfloat16(tensor):
    print(f"Converting tensor from {tensor.dtype} to bfloat16, shape: {tensor.shape}")
    return tensor.to(torch.bfloat16)

def to_fp32(tensor):
    print(f"Converting tensor from {tensor.dtype} to fp32, shape: {tensor.shape}")
    return tensor.to(torch.float32)

def sync_model_dtype(model, target_dtype):
    print(f"Syncing model to {target_dtype}")
    for param in model.parameters():
        param.data = param.data.to(target_dtype)
    return model

device = torch.device("cuda")
model_id = "google/gemma-3-4b-pt"
audio_model_name = "facebook/wav2vec2-base"

batch_size = 40 # A100 80GB - увеличенный размер для лучшего обучения!
num_epochs = 3
learning_rate = 3e-4
weight_decay = 0.0001
max_grad_norm = 5.0  # Gradient clipping
warmup_steps = 100
save_every_steps = 100  # Сохранять каждые 100 шагов
save_latest_every_steps = 10  # Сохранять последний чекпоинт каждые 10 шагов

input_dim = 768
output_dim = 2560

experiment_name = f"audio_projector_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
# checkpoint_dir = f"./checkpoints/{experiment_name}"
# os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_dir = "/home/jovyan/persistent_volume/"

# Инициализация W&B
wandb.init(
    project="audio-projector",
    name=experiment_name,
    config={
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "max_grad_norm": max_grad_norm,
        "input_dim": input_dim,
        "output_dim": output_dim,
        "model_id": model_id,
        "audio_model_name": audio_model_name
    }
)

# Переменные для отслеживания лучших чекпоинтов
best_val_loss = float('inf')
best_checkpoint_path = None
latest_checkpoint_path = None  # Путь к последнему чекпоинту

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

hf_token = os.getenv('HF_TOKEN')
if hf_token:
    login(token=hf_token)

print(f"🚀 Эксперимент: {experiment_name}")
print(f"📁 Чекпоинты: {checkpoint_dir}")
print(f"🖥️  Устройство: {device}")
print(f"⚙️  Конфигурация:")
print(f"   - Batch size: {batch_size}")
print(f"   - Epochs: {num_epochs}")
print(f"   - Learning rate: {learning_rate}")
print(f"   - Weight decay: {weight_decay}")
print(f"   - Gradient clipping: {max_grad_norm}")
print(f"   - Save best every: {save_every_steps} steps")
print(f"   - Save latest every: {save_latest_every_steps} steps")
print(f"🎵 Audio model: {audio_model_name}")
print(f"🤖 LLM: {model_id}")
print(f"🔗 Projector: {input_dim} -> {output_dim}")

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("🔧 Применяем патч для загрузки gemma-3-4b-pt как текстовой модели...")
multi_cfg = AutoConfig.from_pretrained(model_id, token=hf_token)
text_cfg_dict = multi_cfg.text_config.to_dict()

text_cfg_dict["vocab_size"] = 262208

text_cfg_dict.update({"bos_token_id": tokenizer.bos_token_id,
                      "eos_token_id": tokenizer.eos_token_id,
                      "pad_token_id": tokenizer.pad_token_id})

text_cfg = Gemma3TextConfig(**text_cfg_dict)
gemma_model = Gemma3ForCausalLM.from_pretrained(
    model_id,
    config=text_cfg,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    device_map="auto",
    token=hf_token
)
print("✅ Патч успешно применен, модель загружена.")

gemma_model.eval()
for param in gemma_model.parameters():
    param.requires_grad = False

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(audio_model_name)
wav2vec2 = Wav2Vec2Model.from_pretrained(audio_model_name)
wav2vec2 = wav2vec2.to(torch.bfloat16).to(device)
wav2vec2.eval()
for param in wav2vec2.parameters():
    param.requires_grad = False

print(f"Gemma parameters: {sum(p.numel() for p in gemma_model.parameters()):,}")
print(f"Wav2vec2 parameters: {sum(p.numel() for p in wav2vec2.parameters()):,}")

In [ ]:
class AudioProjector(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=None):
        super().__init__()
        # Увеличиваем скрытую размерность для лучшего обучения
        if hidden_dim is None:
            hidden_dim = max(input_dim * 4, output_dim * 2)  # Намного больше!
        
        print(f"🔧 AudioProjector: {input_dim} → {hidden_dim} → {output_dim}")
        
        self.proj = nn.Sequential(
            nn.LayerNorm(input_dim),
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),  # ← Заменили ReLU на GELU для лучших градиентов!
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.GELU(),  # ← И здесь тоже GELU
            nn.Dropout(0.1),
            nn.Linear(hidden_dim // 2, output_dim),
            nn.LayerNorm(output_dim)
        )
        
        # Инициализация весов
        for layer in self.proj:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        original_dtype = x.dtype
        x_fp32 = x.to(torch.float32)
        if next(self.proj.parameters()).dtype != torch.float32:
            self.proj = self.proj.float()
        output_fp32 = self.proj(x_fp32)
        return output_fp32.to(original_dtype)

print("✅ Класс AudioProjector обновлен с GELU активацией и увеличенной размерностью!")
print("🎯 Новая архитектура: LayerNorm → Linear → GELU → Dropout → Linear → GELU → Dropout → Linear → LayerNorm")
print("⚡ Входные данные: любой тип → Вычисления: FP32 → Выход: исходный тип")

In [ ]:
# Создаем улучшенный проектор с GELU и увеличенной размерностью
projector = AudioProjector(input_dim, output_dim).to(device).float()
print(f"🚀 Создан улучшенный AudioProjector:")
print(f"   ✅ GELU активация вместо ReLU")
print(f"   ✅ Увеличенная размерность: {input_dim} → {max(input_dim * 4, output_dim * 2)} → {max(input_dim * 4, output_dim * 2) // 2} → {output_dim}")
print(f"   ✅ Dropout регуляризация: 0.1")
print(f"   ✅ Xavier инициализация весов")

optimizer = optim.AdamW(
    projector.parameters(), 
    lr=learning_rate, 
    weight_decay=weight_decay,
    betas=(0.9, 0.999),
    eps=1e-8
)

scheduler = None

scaler = torch.amp.GradScaler('cuda')
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

prefix = "Audio Transcription: " # или "Транскрипция аудио: "
prefix_ids = tokenizer(prefix, return_tensors="pt").input_ids.to(device)
with torch.no_grad():
    prefix_embeds = gemma_model.get_input_embeddings()(prefix_ids).to(dtype=torch.bfloat16)

print(f"✅ Оптимизатор: AdamW (lr={learning_rate}, wd={weight_decay})")
print(f"✅ Gradient clipping: {max_grad_norm}")
print(f"✅ Mixed precision: включен")
print(f"✅ Префикс промпта: '{prefix}'")

# 🔍 Проверяем архитектуру проектора
print(f"\n🔍 Проверка архитектуры проектора:")
for i, layer in enumerate(projector.proj):
    if hasattr(layer, '__class__'):
        layer_name = layer.__class__.__name__
        if layer_name == 'GELU':
            print(f"   ✅ Слой {i}: {layer_name} (активация GELU найдена!)")
        elif 'Linear' in layer_name:
            print(f"   📦 Слой {i}: {layer_name} ({layer.in_features} → {layer.out_features})")
        else:
            print(f"   🔧 Слой {i}: {layer_name}")
            
total_params = sum(p.numel() for p in projector.parameters())
print(f"📊 Общее количество параметров проектора: {total_params:,}")

In [ ]:
class TrainingLogger:
    def __init__(self, experiment_name, save_dir):
        self.experiment_name = experiment_name
        self.save_dir = save_dir
        self.logs = {
            'step': [],
            'train_loss': [],
            'val_loss': [],
            'val_perplexity': [],
            'val_wer': [],
            'val_bleu': [],
            'val_rouge_l': [],
            'learning_rate': [],
            'grad_norm': []
        }
        
    def log_step(self, step, train_loss, lr, grad_norm=None):
        """Логирование шага обучения"""
        wandb.log({
            'train/loss': train_loss,
            'train/learning_rate': lr,
            'train/grad_norm': grad_norm if grad_norm else 0,
            'step': step
        })
        
    def log_validation(self, step, val_metrics):
        """Логирование валидации"""
        self.logs['step'].append(step)
        self.logs['val_loss'].append(val_metrics['loss'])
        self.logs['val_perplexity'].append(val_metrics['perplexity'])
        self.logs['val_wer'].append(val_metrics['wer'])
        self.logs['val_bleu'].append(val_metrics['bleu'])
        self.logs['val_rouge_l'].append(val_metrics['rouge_l'])
        
        wandb.log({
            'val/loss': val_metrics['loss'],
            'val/perplexity': val_metrics['perplexity'],
            'val/wer': val_metrics['wer'],
            'val/bleu': val_metrics['bleu'],
            'val/rouge_l': val_metrics['rouge_l'],
            'step': step
        })
    
    def plot_training_curves(self):
        if len(self.logs['step']) == 0:
            return
            
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle(f'Training Progress: {self.experiment_name}', fontsize=16, fontweight='bold')
        
        axes[0, 0].plot(self.logs['step'], self.logs['val_loss'], 'r-', label='Val Loss', linewidth=2)
        axes[0, 0].set_xlabel('Step')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].set_title('Validation Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        axes[0, 1].plot(self.logs['step'], self.logs['val_perplexity'], 'g-', linewidth=2)
        axes[0, 1].set_xlabel('Step')
        axes[0, 1].set_ylabel('Perplexity')
        axes[0, 1].set_title('Validation Perplexity')
        axes[0, 1].grid(True, alpha=0.3)
        
        axes[0, 2].plot(self.logs['step'], self.logs['val_wer'], 'orange', linewidth=2)
        axes[0, 2].set_xlabel('Step')
        axes[0, 2].set_ylabel('WER')
        axes[0, 2].set_title('Word Error Rate')
        axes[0, 2].grid(True, alpha=0.3)
        
        axes[1, 0].plot(self.logs['step'], self.logs['val_bleu'], 'purple', linewidth=2)
        axes[1, 0].set_xlabel('Step')
        axes[1, 0].set_ylabel('BLEU Score')
        axes[1, 0].set_title('BLEU Score')
        axes[1, 0].grid(True, alpha=0.3)
        
        axes[1, 1].plot(self.logs['step'], self.logs['val_rouge_l'], 'brown', linewidth=2)
        axes[1, 1].set_xlabel('Step')
        axes[1, 1].set_ylabel('ROUGE-L')
        axes[1, 1].set_title('ROUGE-L Score')
        axes[1, 1].grid(True, alpha=0.3)
        
        # Пустой график для симметрии
        axes[1, 2].axis('off')
        
        plt.tight_layout()
        
        plot_path = os.path.join(self.save_dir, f'training_curves_step_{self.logs["step"][-1]}.png')
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"📊 График сохранен: {plot_path}")
        plt.show()
    
    def save_logs(self):
        if len(self.logs['step']) == 0:
            return None
        df = pd.DataFrame(self.logs)
        csv_path = os.path.join(self.save_dir, 'validation_logs.csv')
        df.to_csv(csv_path, index=False)
        print(f"📝 Логи валидации сохранены: {csv_path}")
        return df

def process_batch(batch, model, projector, wav2vec2, tokenizer, prefix_embeds, device):
    input_values = batch["input_values"].to(device, dtype=torch.bfloat16)
    input_ids = batch["input_ids"].to(device)

    with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu', dtype=torch.bfloat16):
        audio_embeds = wav2vec2(input_values).last_hidden_state.mean(dim=1)
        projected_audio = projector(audio_embeds)
        batch_prefix_embeds = prefix_embeds.expand(projected_audio.size(0), -1, -1)
        
        prompt_embeds = torch.cat([batch_prefix_embeds, projected_audio.unsqueeze(1)], dim=1)
        
        embedding_input_ids = input_ids.clone()
        embedding_input_ids[embedding_input_ids == -100] = tokenizer.pad_token_id
        target_embeds = model.get_input_embeddings()(embedding_input_ids)

        inputs_embeds = torch.cat([prompt_embeds, target_embeds], dim=1)
        
        prompt_len = prompt_embeds.shape[1]
        prompt_labels = torch.full((projected_audio.size(0), prompt_len), -100, device=device, dtype=torch.long)
        labels = torch.cat([prompt_labels, input_ids], dim=1)

        outputs = model(inputs_embeds=inputs_embeds, labels=labels)
        
    return outputs, prompt_embeds

def evaluate_with_metrics(model, projector, wav2vec2, dataloader, tokenizer, prefix_embeds, device):
    model.eval()
    projector.eval()
    wav2vec2.eval()
    total_loss, total_wer, total_bleu = 0.0, 0.0, 0.0
    total_rouge_1, total_rouge_2, total_rouge_l = 0.0, 0.0, 0.0
    count = 0
    debug_count = 0  # Счетчик для отладочных принтов
    smooth = SmoothingFunction().method1
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    print("🔍 Начинаем валидацию с отладочными принтами...")
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="🔍 Validation"):
            input_ids = batch["input_ids"].to(device)

            outputs, prompt_embeds = process_batch(
                batch, model, projector, wav2vec2, tokenizer, prefix_embeds, device
            )
            loss = outputs.loss
            total_loss += loss.item()

            with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu', dtype=torch.bfloat16):
                generated_ids = model.generate(
                    inputs_embeds=prompt_embeds,
                    max_new_tokens=30,  # ← УМЕНЬШИЛИ с 100 до 30!
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.pad_token_id,
                    do_sample=False
                )
            
            input_len = prompt_embeds.shape[1]
            generated_ids_only = generated_ids[:, input_len:]

            for j in range(generated_ids.size(0)):
                pred_text = tokenizer.decode(generated_ids_only[j], skip_special_tokens=True).strip()
                ref_text_ids = input_ids[j]
                ref_text_ids = ref_text_ids[ref_text_ids != -100]
                ref_text = tokenizer.decode(ref_text_ids, skip_special_tokens=True).strip()
                
                if ref_text and pred_text:
                    current_wer = jiwer.wer(ref_text, pred_text)
                    current_bleu = sentence_bleu([ref_text.split()], pred_text.split(), smoothing_function=smooth)
                    rouge_scores = rouge_scorer_obj.score(ref_text, pred_text)
                    
                    total_wer += current_wer
                    total_bleu += current_bleu
                    total_rouge_1 += rouge_scores['rouge1'].fmeasure
                    total_rouge_2 += rouge_scores['rouge2'].fmeasure
                    total_rouge_l += rouge_scores['rougeL'].fmeasure
                    count += 1
                    
                    # 🔍 ОТЛАДОЧНЫЕ ПРИНТЫ (показываем первые 3 примера)
                    if debug_count < 3:
                        print(f"\n📝 Пример {debug_count + 1}:")
                        print(f"   🎯 Эталон:    '{ref_text}'")
                        print(f"   🤖 Генерация: '{pred_text}'")
                        print(f"   📊 WER: {current_wer:.3f}, BLEU: {current_bleu:.3f}")
                        print(f"   💡 WER {current_wer:.3f} = {current_wer*100:.1f}% ошибок")
                        debug_count += 1
                    
    avg_loss = total_loss / len(dataloader)
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    avg_wer = total_wer / count if count > 0 else 0.0
    avg_bleu = total_bleu / count if count > 0 else 0.0
    avg_rouge_1 = total_rouge_1 / count if count > 0 else 0.0
    avg_rouge_2 = total_rouge_2 / count if count > 0 else 0.0
    avg_rouge_l = total_rouge_l / count if count > 0 else 0.0
    
    # 📊 УЛУЧШЕННЫЙ ИТОГОВЫЙ ПРИНТ
    print(f"\n📊 Результаты валидации ({count} примеров):")
    print(f"   📉 Loss: {avg_loss:.4f}")
    print(f"   🎯 WER: {avg_wer:.4f} (это доля, не %) = {avg_wer*100:.1f}% ошибок")
    print(f"   📝 BLEU: {avg_bleu:.4f}")
    print(f"   🔍 ROUGE-1: {avg_rouge_1:.4f}")
    
    return {
        'loss': avg_loss, 'perplexity': perplexity,
        'wer': avg_wer, 'bleu': avg_bleu,
        'rouge_1': avg_rouge_1, 'rouge_2': avg_rouge_2, 'rouge_l': avg_rouge_l
    }

In [ ]:
from torch.utils.data import DataLoader
import time

def collate_fn(batch):
    t0 = time.time()
    input_values = [item['input_values'] for item in batch]
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    input_values = pad_sequence(input_values, batch_first=True)
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=-100)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    return {
        'input_values': input_values,
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }

class AudioTextDataset(Dataset):
    def __init__(self, data, tokenizer, feature_extractor, zip_path=None):
        self.data = data
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.zip_file = None
        self.zip_manifest = None  # Кэш для быстрого доступа к файлам в ZIP
        
        if zip_path and os.path.exists(zip_path):
            try:
                self.zip_file = zipfile.ZipFile(zip_path, 'r')
                print(f"📦 ZIP-файл открыт: {zip_path}")
                
                # Создаем манифест (словарь) для мгновенного поиска путей
                print("⚡️ Создание манифеста ZIP-файла для ускорения доступа...")
                self.zip_manifest = {
                    p: p
                    for p in self.zip_file.namelist()
                    if p.lower().endswith(('.flac', '.wav', '.mp3'))
                }
                print(f"✅ Манифест создан: {len(self.zip_manifest)} аудиофайлов.")

            except Exception as e:
                print(f"⚠️ Ошибка открытия или чтения ZIP: {e}")
                self.zip_file = None
        else:
            print(f"⚠️ ZIP файл не найден: {zip_path}")
            
        print(f"📊 Датасет содержит: {len(self.data)} записей")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        t0 = time.time()
        item = self.data[idx]
        audio_path = item["audio_path"]
        speaker_text = item["speaker_text"]
        
        # Загрузка аудио
        try:
            if self.zip_file and self.zip_manifest is not None:
                # Используем манифест для прямого доступа по относительному пути
                found_path = self.zip_manifest.get(audio_path)
                if found_path:
                    with self.zip_file.open(found_path) as audio_file:
                        audio_data = audio_file.read()
                        waveform, sr = torchaudio.load(io.BytesIO(audio_data))
                else:
                    raise FileNotFoundError(f"Файл '{audio_path}' не найден в манифесте ZIP.")
            else:
                # Загрузка с диска
                waveform, sr = torchaudio.load(audio_path)
                
        except Exception as e:
            print(f"⚠️ Ошибка загрузки {audio_path}: {e}")
            # Возвращаем пустой аудио для пропуска
            waveform = torch.zeros(1, 16000)
            sr = 16000
        
        # Обработка аудио
        if sr != self.feature_extractor.sampling_rate:
            waveform = torchaudio.functional.resample(waveform, sr, self.feature_extractor.sampling_rate)
        
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        
        # Извлечение признаков
        inputs = self.feature_extractor(
            waveform.squeeze().numpy(),
            sampling_rate=self.feature_extractor.sampling_rate,
            return_tensors="pt"
        )
        
        # Токенизация текста
        tokens = self.tokenizer(
            speaker_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        return {
            "input_values": inputs.input_values.squeeze(0),
            "input_ids": tokens.input_ids.squeeze(0),
            "attention_mask": tokens.attention_mask.squeeze(0)
        }
    
    def __del__(self):
        if hasattr(self, 'zip_file') and self.zip_file:
            self.zip_file.close()

print("✅ Классы Dataset и collate_fn определены")
print("💡 DataLoader будут созданы после определения датасетов в следующих ячейках")

In [ ]:
jsonl_path = "transcripts.jsonl"
zip_path = "LibriSpeech.zip"  # Обновленный путь к ZIP-файлу

resume_batches = 0  # Укажите число батчей, которые хотите пропустить при возобновлении

print(f"📂 Загружаем данные из {jsonl_path}...")

with open(jsonl_path, "r", encoding="utf-8") as f:
    all_data = [json.loads(line) for line in f]

print(f"📊 Загружено записей: {len(all_data)}")

# Нормализация данных
normalized_data = []
for item in all_data:
    normalized_item = {
        "audio_path": item.get("audio_filepath", ""),
        "speaker_text": item.get("text", ""),
        "language": item.get("language", "en"),
        "source": item.get("source", "unknown")
    }
    normalized_data.append(normalized_item)

# Разбиваем на train/val
total_records = len(normalized_data)
train_data, val_data = train_test_split(normalized_data, test_size=0.1, random_state=42)

# Пропускаем первые resume_batches батчей в train_data
if resume_batches > 0:
    skip_samples = resume_batches * batch_size
    train_data = train_data[skip_samples:]
    print(f"🚀 Пропускаем первые {skip_samples} примеров (resume_batches={resume_batches})")

# Создаем уменьшенный validation subset для быстрой валидации
# Берем только 1/8 от validation data (это примерно 1.25% от всех данных)
val_subset_size = max(50, len(val_data) // 8)
val_subset_data = random.sample(val_data, val_subset_size)

print(f"📊 Размеры данных:")
print(f"   - Train: {len(train_data)} примеров")
print(f"   - Val (полный): {len(val_data)} примеров")
print(f"   - Val (subset): {len(val_subset_data)} примеров")
print(f"   - Ускорение валидации: ~{len(val_data) // len(val_subset_data)}x")

# Создание датасетов
train_dataset = AudioTextDataset(train_data, tokenizer, feature_extractor, zip_path=zip_path)
val_dataset = AudioTextDataset(val_subset_data, tokenizer, feature_extractor, zip_path=zip_path)

# === ЛОГИКА ПРОПУСКА ДАННЫХ ДЛЯ ВОЗОБНОВЛЕНИЯ ОБУЧЕНИЯ ===
resume_step = 0  # Укажите с какого шага продолжить (0 = с начала)

if resume_step > 0:
    # 🔥 РЕАЛЬНО пропускаем данные в датасете
    skip_samples = resume_step * batch_size
    original_len = len(train_dataset.data)
    
    if skip_samples < original_len:
        train_dataset.data = train_dataset.data[skip_samples:]
        print(f"🚀 Пропустили {skip_samples} примеров в датасете")
        print(f"   Было: {original_len}, стало: {len(train_dataset.data)}")
    else:
        print(f"⚠️ Пропуск {skip_samples} примеров больше размера датасета {original_len}!")
        print("   Начинаем с начала следующей эпохи")
        resume_step = 0
else:
    print("🌆 Начинаем обучение с начала")

# Создание DataLoader после определения датасетов
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0,  # Отключаем многопроцессность для стабильности
    pin_memory=False # Отключаем pin_memory для избежания ошибок
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,  # Отключаем многопроцессность для стабильности
    pin_memory=False # Отключаем pin_memory для избежания ошибок
)

print(f"📊 DataLoader готовы: {len(train_loader)} train batches, {len(val_loader)} val batches")

# 🔧 ИСПРАВЛЯЕМ ПРОБЛЕМУ С УСТРОЙСТВАМИ
print("🔧 Переносим все модели на CUDA...")
wav2vec2 = wav2vec2.to(device)
projector = projector.to(device)
gemma_model = gemma_model.to(device)
print("✅ Все модели на CUDA")

# Проверяем устройства
print(f"📍 wav2vec2 device: {next(wav2vec2.parameters()).device}")
print(f"📍 projector device: {next(projector.parameters()).device}")
print(f"📍 gemma_model device: {next(gemma_model.parameters()).device}")

total_steps = num_epochs * len(train_loader)
scheduler = OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    total_steps=total_steps,
    pct_start=0.1,
    anneal_strategy='cos'
)

print(f"📅 Общее количество шагов: {total_steps}")
print(f"🔥 Warmup шагов: {int(0.1 * total_steps)}")

logger = TrainingLogger(experiment_name, checkpoint_dir)

# === ФУНКЦИИ ДЛЯ РАБОТЫ С ЧЕКПОИНТАМИ ===

def find_latest_checkpoint(checkpoint_dir):
    """Находит самый последний (по времени) чекпоинт."""
    pattern = os.path.join(checkpoint_dir, "latest_checkpoint_*.pt")    
    checkpoints = glob.glob(pattern) 
    
    return max(checkpoints, key=os.path.getctime) if checkpoints else None

def find_best_checkpoint(checkpoint_dir):
    """Находит лучший чекпоинт по val_loss."""
    pattern = os.path.join(checkpoint_dir, "best_checkpoint_*.pt")
    checkpoints = glob.glob(pattern)
    return checkpoints[0] if checkpoints else None

def load_checkpoint(path, projector, optimizer, scheduler):
    """Загружает состояние из чекпоинта для возобновления обучения."""
    global best_val_loss
    print(f"🔄 Загрузка чекпоинта: {path}")
    checkpoint = torch.load(path, map_location=device)
    
    projector.load_state_dict(checkpoint['projector_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    start_epoch = checkpoint['epoch']
    global_step = checkpoint['step']
    best_val_loss = checkpoint.get('best_val_loss', float('inf'))
    
    print(f"✅ Возобновление с эпохи {start_epoch}, шаг {global_step}. Лучший val_loss: {best_val_loss:.4f}")
    return start_epoch, global_step

def save_checkpoint(step, epoch, is_best=False):
    """Сохранение чекпоинта"""
    global best_checkpoint_path
    
    checkpoint_data = {
        'step': step,
        'epoch': epoch,
        'projector_state_dict': projector.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_loss': best_val_loss,
        'config': {
            'learning_rate': learning_rate,
            'weight_decay': weight_decay,
            'max_grad_norm': max_grad_norm,
            'batch_size': batch_size,
            'input_dim': input_dim,
            'output_dim': output_dim
        }
    }
    
    if is_best:
        # Удаляем предыдущий лучший чекпоинт
        if best_checkpoint_path and os.path.exists(best_checkpoint_path):
            os.remove(best_checkpoint_path)
        
        best_checkpoint_path = os.path.join(checkpoint_dir, f"best_checkpoint_step_{step}.pt")
        torch.save(checkpoint_data, best_checkpoint_path)
        print(f"🏆 Лучший чекпоинт сохранен: {best_checkpoint_path}")
    else:
        # Обычный чекпоинт
        checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_step_{step}.pt")
        torch.save(checkpoint_data, checkpoint_path)
        print(f"💾 Чекпоинт сохранен: {checkpoint_path}")

def save_latest_checkpoint(step, epoch):
    """Сохранение последнего чекпоинта с перезаписью"""
    global latest_checkpoint_path
    
    checkpoint_data = {
        'step': step,
        'epoch': epoch,
        'projector_state_dict': projector.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_loss': best_val_loss,
        'config': {
            'learning_rate': learning_rate,
            'weight_decay': weight_decay,
            'max_grad_norm': max_grad_norm,
            'batch_size': batch_size,
            'input_dim': input_dim,
            'output_dim': output_dim
        }
    }
    
    # Удаляем предыдущий последний чекпоинт
    if latest_checkpoint_path and os.path.exists(latest_checkpoint_path):
        os.remove(latest_checkpoint_path)
    
    latest_checkpoint_path = os.path.join(checkpoint_dir, f"latest_checkpoint_epoch_{epoch}_step_{step}.pt")
    torch.save(checkpoint_data, latest_checkpoint_path)
    print(f"📄 Последний чекпоинт: epoch_{epoch}_step_{step}")

# === ЛОГИКА ВОЗОБНОВЛЕНИЯ ОБУЧЕНИЯ ===

# Установите resume_training = True для возобновления с последнего чекпоинта
resume_training = True
start_epoch = 0
global_step = 0

if resume_training:
    checkpoint_path = find_latest_checkpoint(checkpoint_dir)
    # Или можно выбрать лучший: checkpoint_path = find_best_checkpoint(checkpoint_dir)
    
    if checkpoint_path:
        checkpoint_epoch, global_step = load_checkpoint(checkpoint_path, projector, optimizer, scheduler)
        # Переводим номер сохранённой эпохи в индекс цикла (0-based)
        start_epoch = checkpoint_epoch - 1
        print(f"🔄 Продолжаем обучение с шага {global_step}, эпохи {checkpoint_epoch}")
    else:
        print("⚠️ Чекпоинты не найдены. Начинаем новое обучение.")
        resume_training = False

print(f"🚀 Начинаем обучение Audio Projector!")
print(f"📈 W&B проект: {wandb.run.project} / {wandb.run.name}")
print(f"🔄 Режим: {'Возобновление' if resume_training else 'Новое обучение'}")

# === ОСНОВНОЙ ЦИКЛ ОБУЧЕНИЯ ===

for epoch in range(start_epoch, num_epochs):
    print(f"\n{'='*50}")
    print(f"🔄 ЭПОХА {epoch+1}/{num_epochs}")
    print(f"{'='*50}")
    
    projector.train()
    wav2vec2.eval()
    gemma_model.eval()
    
    # ПРАВИЛЬНЫЙ пропуск батчей: используем enumerate и continue
    is_resumed_epoch = resume_training and epoch == start_epoch
    batches_to_skip = (global_step % len(train_loader)) if is_resumed_epoch else 0
    
    if batches_to_skip > 0:
        print(f"⏭️  Пропускаем {batches_to_skip} батчей в эпохе {epoch+1}")
        print(f"⚡ НЕ загружая данные для пропущенных батчей...")

    # Настраиваем tqdm для отображения правильного прогресса
    progress_bar = tqdm(
        enumerate(train_loader), 
        total=len(train_loader),
        initial=batches_to_skip,  # Начинаем отображение с нужного места
        desc=f"Epoch {epoch+1}"
    )

    # Флаг для одноразового лога первого батча
    first_batch_logged = False

    for batch_idx, batch in progress_bar:
        # После урезания датасета batch_idx соответствует реальному номеру
        real_batch_number = global_step + batch_idx
        
        # Показываем информацию о первом обрабатываемом батче
        if not first_batch_logged:
            print(f"\n✅ Начинаем реальную обработку с батча {batch_idx} (глобальный шаг {real_batch_number})")
            print(f"   Размер аудио-тензора: {batch['input_values'].shape}")
            print(f"   Размер текстового-тензора: {batch['input_ids'].shape}")
            first_batch_logged = True
                
        # Обновляем global_step для текущего батча
        current_global_step = real_batch_number
            
        optimizer.zero_grad()
        
        outputs, _ = process_batch(
            batch, gemma_model, projector, wav2vec2, tokenizer, prefix_embeds, device
        )
        loss = outputs.loss
                
        # Backward pass
        scaler.scale(loss).backward()
            
        scaler.unscale_(optimizer)
        grad_norm = torch.nn.utils.clip_grad_norm_(projector.parameters(), max_grad_norm)
        
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        current_lr = scheduler.get_last_lr()[0]
        
        # Логирование в W&B - используем правильный global_step
        logger.log_step(current_global_step, loss.item(), current_lr, grad_norm.item())
        
        # Обновление прогресс-бара
        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'LR': f'{current_lr:.2e}',
            'Step': current_global_step
        })
        
        # Сохранение последнего чекпоинта каждые 10 шагов
        if current_global_step % save_latest_every_steps == 0:
            save_latest_checkpoint(current_global_step, epoch + 1)
            print(f"💾 Сохранен latest checkpoint на шаге {current_global_step}")
        
        # Сохранение и валидация каждые 100 шагов
        if current_global_step % save_every_steps == 0:
            print(f"\n🔍 Валидация на шаге {current_global_step}...")
            
            val_metrics = evaluate_with_metrics(
                gemma_model, projector, wav2vec2, val_loader, 
                tokenizer, prefix_embeds, device
            )
            
            logger.log_validation(current_global_step, val_metrics)
            
            print(f"📊 Результаты валидации (шаг {current_global_step}):")
            print(f"   Loss: {val_metrics['loss']:.4f}")
            print(f"   Perplexity: {val_metrics['perplexity']:.2f}")
            print(f"   WER: {val_metrics['wer']:.3f}")
            print(f"   BLEU: {val_metrics['bleu']:.3f}")
            print(f"   ROUGE-L: {val_metrics['rouge_l']:.3f}")
            
            # Сохранение чекпоинта
            is_best = val_metrics['loss'] < best_val_loss
            if is_best:
                best_val_loss = val_metrics['loss']
                print(f"🏆 Новый лучший результат! Loss: {best_val_loss:.4f}")
            
            save_checkpoint(current_global_step, epoch + 1, is_best)
            logger.plot_training_curves()
            
            # Очистка памяти после валидации
            del val_metrics
            torch.cuda.empty_cache()
            
            # Возвращаемся в train режим
            projector.train()
    
    # ВАЖНО: После завершения первой возобновленной эпохи сбрасываем флаг
    if is_resumed_epoch:
        resume_training = False
        print(f"✅ Эпоха {epoch+1} завершена, переходим к обычному режиму")

print(f"\n{'='*50}")
print(f"🎉 ОБУЧЕНИЕ ЗАВЕРШЕНО!")
print(f"{'='*50}")

# Финальная валидация на полном validation set
print("🔍 Финальная валидация на полном validation set...")
print("💡 Это займет больше времени, но даст более точную оценку")

# Создаем полный validation dataset для финальной оценки
full_val_dataset = AudioTextDataset(val_data, tokenizer, feature_extractor, zip_path=zip_path)
full_val_loader = DataLoader(full_val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

final_val_metrics = evaluate_with_metrics(
    gemma_model, projector, wav2vec2, full_val_loader, 
    tokenizer, prefix_embeds, device
)

print(f"\n📊 Финальные результаты (полный validation set, {len(full_val_dataset)} примеров):")
print(f"   Loss: {final_val_metrics['loss']:.4f}")
print(f"   Perplexity: {final_val_metrics['perplexity']:.2f}")
print(f"   WER: {final_val_metrics['wer']:.3f}")
print(f"   BLEU: {final_val_metrics['bleu']:.3f}")
print(f"   ROUGE-L: {final_val_metrics['rouge_l']:.3f}")

logger.log_validation(global_step, final_val_metrics)

# Очистка памяти
del final_val_metrics, full_val_dataset, full_val_loader
torch.cuda.empty_cache()

logger.plot_training_curves()
final_logs_df = logger.save_logs()

if final_logs_df is not None:
    print(f"\n📋 Финальная статистика:")
    print(final_logs_df.tail().round(4))

# Сохранение финальной модели
final_model_path = os.path.join(checkpoint_dir, "final_projector.pt")
torch.save(projector.state_dict(), final_model_path)
print(f"🏆 Финальная модель: {final_model_path}")

# Завершение работы Weights & Biases
wandb.finish()
print("🏁 wandb завершён корректно.")

# Очистка памяти и сборка мусора
import gc
torch.cuda.empty_cache()
gc.collect()
print("✨ Оперативная память очищена")

In [ ]:
def test_random_sample(dataset, original_data, model, projector, wav2vec2, tokenizer, prefix_embeds, device):
    """Берет случайный пример из датасета, генерирует текст и воспроизводит аудио."""
    model.eval()
    projector.eval()

    # 1. Берем случайный пример
    idx = random.randint(0, len(dataset) - 1)
    original_sample_info = original_data[idx]
    
    print(f"--- 🧪 Тестируем модель на примере #{idx} ---")
    print(f"📄 Файл: {original_sample_info['audio_path']}")

    # 2. Загружаем и обрабатываем аудио (логика из датасета)
    audio_path = original_sample_info['audio_path']
    zip_file = getattr(dataset, 'zip_file', None)
    waveform = None
    sr = 16000

    try:
        if zip_file:
            found_path = next((p for p in zip_file.namelist() if p.endswith(os.path.basename(audio_path))), None)
            if found_path:
                with zip_file.open(found_path) as audio_file:
                    waveform, sr = torchaudio.load(io.BytesIO(audio_file.read()))
            else:
                raise FileNotFoundError(f"Аудио не найдено в ZIP: {audio_path}")
        elif os.path.exists(audio_path):
            waveform, sr = torchaudio.load(audio_path)
        else:
            raise FileNotFoundError(f"Аудио не найдено на диске: {audio_path}")
    except Exception as e:
        print(f"❌ Ошибка загрузки аудио: {e}")
        return

    # 3. Генерируем транскрипцию
    with torch.no_grad():
        # Предобработка аудио
        if sr != feature_extractor.sampling_rate:
            waveform = torchaudio.functional.resample(waveform, sr, feature_extractor.sampling_rate)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        
        input_values = feature_extractor(waveform.squeeze().numpy(), sampling_rate=feature_extractor.sampling_rate, return_tensors="pt").input_values
        input_values = input_values.to(device, dtype=torch.bfloat16)

        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            audio_embeds = wav2vec2(input_values).last_hidden_state.mean(dim=1)
            projected_audio = projector(audio_embeds)
            batch_prefix_embeds = prefix_embeds.expand(projected_audio.size(0), -1, -1)
            prompt_embeds = torch.cat([batch_prefix_embeds, projected_audio.unsqueeze(1)], dim=1)

            generated_ids = model.generate(
                inputs_embeds=prompt_embeds, max_new_tokens=100,
                eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
                do_sample=True, top_k=50, top_p=0.95
            )
            
    generated_text = tokenizer.decode(generated_ids[0, prompt_embeds.shape[1]:], skip_special_tokens=True).strip()
    reference_text = original_sample_info['speaker_text']

    # 4. Выводим результат
    print(f"\n🗣️  Оригинальный текст:")
    print(f"    '{reference_text}'")
    print(f"\n🤖  Результат модели:")
    print(f"    '{generated_text}'")
    
    # 5. Простая оценка качества
    if reference_text and generated_text:
        wer_score = jiwer.wer(reference_text, generated_text)
        print(f"\n📊 WER (Word Error Rate): {wer_score:.3f}")
        if wer_score < 0.3:
            print("✅ Отличный результат!")
        elif wer_score < 0.5:
            print("👍 Хороший результат")
        elif wer_score < 0.8:
            print("⚠️ Средний результат")
        else:
            print("❌ Требует улучшения")
    
    # 6. Воспроизводим аудио
    print(f"\n🎵 Воспроизведение аудио ({waveform.shape[1]/sr:.1f} сек):")
    display(Audio(waveform.numpy(), rate=sr))

def collate_fn(batch):
    # Логи отключены для скорости
    input_values = [item['input_values'] for item in batch]
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    # Применяем pad_sequence без логов
    input_values = pad_sequence(input_values, batch_first=True)
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=-100)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    return {
        'input_values': input_values,
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }

class AudioTextDataset(Dataset):
    def __getitem__(self, idx):
        # Логи отключены для скорости
        item = self.data[idx]
        audio_path = item['audio_path']
        speaker_text = item['speaker_text']
        try:
            if self.zip_file and self.zip_manifest is not None:
                found_path = self.zip_manifest.get(audio_path)
                if found_path:
                    with self.zip_file.open(found_path) as audio_file:
                        audio_data = audio_file.read()
                        waveform, sr = torchaudio.load(io.BytesIO(audio_data))
                else:
                    raise FileNotFoundError
            else:
                waveform, sr = torchaudio.load(audio_path)
        except:
            waveform = torch.zeros(1, 16000)
            sr = 16000
        # Обработка аудио
        if sr != self.feature_extractor.sampling_rate:
            waveform = torchaudio.functional.resample(waveform, sr, self.feature_extractor.sampling_rate)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        inputs = self.feature_extractor(
            waveform.squeeze().numpy(),
            sampling_rate=self.feature_extractor.sampling_rate,
            return_tensors='pt'
        )
        tokens = self.tokenizer(
            speaker_text,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=512
        )
        return {
            'input_values': inputs.input_values.squeeze(0),
            'input_ids': tokens.input_ids.squeeze(0),
            'attention_mask': tokens.attention_mask.squeeze(0)
        }
    def __del__(self):
        if hasattr(self, 'zip_file') and self.zip_file:
            self.zip_file.close()

print("🧪 Для тестирования модели выполните:")
print("test_random_sample(val_dataset, val_data, gemma_model, projector, wav2vec2, tokenizer, prefix_embeds, device)")
print("\n💡 Или загрузите лучший чекпоинт перед тестированием:")
print("# best_path = find_best_checkpoint(checkpoint_dir)")
print("# if best_path: load_checkpoint(best_path, projector, optimizer, scheduler)")
print("# test_random_sample(val_dataset, val_data, gemma_model, projector, wav2vec2, tokenizer, prefix_embeds, device)")